<a href="https://colab.research.google.com/github/Jisang-hwang93/BOK_Report_Team1/blob/master/BoK_Project_Team1_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 수집**

## **1. 한국은행 금융통화위원회 의사록 다운로드**
본 논문에서 사용된 금융통화위원회 의사록의 기간은 **2005년 5월부터 2017년 12월까지**임.

**하지만, 해당 코드로는 2005년 8월 9일까지 밖에 다운로드 되지 않음.**

이 기간의 의사록을 다운로드하기 위해

http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085857&fileSn=2 - **2005년 6월 9일 의사록**

http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085858&fileSn=1 - **2005년 5월 12일 의사록**

두 개의 의사록을 **직접 다운받을 것**을 권함.

### **1-1. 의사록 다운로드 경로 설정**
**의사록 pdf 파일, 원제목, 업로드 일자 리스트에 저장**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# 빈 리스트 생성
bok = []
bok_title = []

# 31번 페이지까지의 pdf파일 경로
for i in range(1, 32):
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=' + str(i)
    response = requests.get(url)
    response.encoding = 'utf-8' # 한글이므로 encoding을 utf-8로 지정
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # pdf 파일 저장
    for j in soup.select("div.fileGoupBox li"):
        # title의 확장자가 'pdf'인 파일만 저장
        if 'pdf' in j.select("a")[0].attrs['title']:
            bok.append(j.select("a")[0].attrs['href'])
    # pdf 파일의 원제목
    for i in soup.select('li div.row span.titlesub'):
        bok_title.append(i.text)

# pdf 다운로드 주소
bok_pdf = []
for file_url in bok:
    bok_pdf.append('http://www.bok.or.kr' + file_url)

# 파일 개수 확인
len(bok_pdf), len(bok_title)

### **1-2. pdf 저장 폴더 설정**
pdf를 저장할 폴더 생성

In [ ]:
mkdir bok_pdf

In [ ]:
# bok_pdf 폴더에 pdf 저장
for num, j in enumerate(bok_pdf):
    url = j
    r = requests.get(url, stream=True)
    # bok_0.pdf 형식으로 저장
    with open('/content/bok_pdf/{}.pdf'.format(bok_title[num]), 'wb') as f:
        f.write(r.content)

### **1-3. pdf를 csv로 변환**

**tika 설치**
tika가 pdfminerer에 비해 더 빠름

In [ ]:
!pip install tika

In [ ]:
from tika import parser
import csv
from tqdm import tqdm

file = open("tika_bok_text.csv", mode="w", encoding="utf-8")
writer = csv.writer(file)
texts=[]

for i in tqdm(range(310)):
    try: 
        raw = parser.from_file('/content/bok_pdf/{}.pdf'.format(bok_title[i]))
        text = [raw['content']] 
        texts.append(text)
    except:
        text = ['error']
        texts.append(text)
        print(i)
    writer.writerow(text)
        
file.close()

### **1-1-4. 의사록 데이터 확인**

In [ ]:
import pandas as pd

# 데이터프레임 형식으로 불러오기
bok_text = pd.read_csv('/content/tika_bok_text.csv', header = None, sep = ',')
bok_text = pd.DataFrame(bok_text)
bok_text.columns=['내용']

# bok_text 확인
bok_text.head()

In [ ]:
import re
from datetime import datetime

bok_text['제목'] = bok_title
date_time = []

for i in bok_text['제목']:
    tmp = re.findall("(?<=\)\().+\..+\.[0-9]+(?=\.?\))", i)
    date_time.append(datetime.strptime(tmp[0], '%Y.%m.%d'))

date_time[0]

In [ ]:
# 날짜, 제목 항목 추가
bok_text['날짜'] = bok_date

bok_text

In [ ]:
bok_text.to_csv('/content/tika_bok_text.csv', encoding='utf-8-sig')

## **2. 채권 리포트 다운로드**

### **2-1. 채권 리포트 다운로드 경로 설정**
**채권 리포트의 pdf 파일, 원제목, 업로드 일자 리스트에 저장**

In [ ]:
report_url = []
report_title = []
report_date = []

# 다운로드 가능한 최대 범위 105페이지까지의 주소
for page in range(1,105):
    res = requests.get('https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2004-01-01&writeToDate=2017-12-31&x=22&y=24&page={}'.format(page))
    bs = BeautifulSoup(res.text, 'html.parser')
    # pdf 파일 저장 경로
    for i in bs.select("td.file"):
        report_url.append(i.select_one("a").attrs['href'])
    
    bs1 = bs.select_one('table[summary="채권분석 리포트 게시판 글목록"]')
    # pdf 파일의 원제목
    for i in bs1.select("td[style='padding-left:10px']"):
        report_title.append(i.text)
    # pdf 파일의 업로드 날짜
    for j in bs.select("td[style='padding-left:5px']"):
        report_date.append(j.text)

# 총 파일의 개수
len(report_url), len(report_title), len(report_date)

### **2-2. pdf 저장 폴더 설정**

In [ ]:
mkdir bond_report

In [ ]:
# pdf 파일 저장
for i, url in enumerate(report_url):
    r = requests.get(url, stream=True)
    with open('/content/bond_report/bond_report{}.pdf'.format(i), 'wb') as f:
        f.write(r.content)

### **2-3. pdf를 text로 변환**

In [ ]:
from tika import parser
import csv
from tqdm import tqdm

file = open("tika_bond_text.csv", mode="w", encoding="utf-8")
writer = csv.writer(file)
texts=[]

for i in tqdm(range(3097)):
    try: 
        raw = parser.from_file('/content/bond_report/bond_report{}.pdf''.format(i))
        text = [raw['content']] 
        texts.append(text)
    except:
        text = ['error']
        texts.append(text)
        print(i)
    writer.writerow(text)
        
file.close()

### **2-4. 채권 리포트 데이터**

In [ ]:
# 데이터프레임 형식으로 불러오기
bond_text = pd.read_csv('/content/tika_bond_all.csv', header = None, sep = ',')
bond_text = pd.DataFrame(bond_text)
bond_text.columns=['내용']

# bond_text 확인
bond_text

In [ ]:
# 날짜, 제목 항목 추가
bond_text['제목'] = bond_title
bond_text['날짜'] = bond_date

bond_text

In [ ]:
# 채권 리포트 데이터프레임 형식으로 저장
bond_text.to_csv('/content/tika_bond_all.csv', encoding='utf-8-sig')

## **3. 금리 데이터 수집**

### **3-1. 콜 금리 수집**

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime

import requests

# 빈 리스트 생성
call_tem = []
call_rate = []
date = []

# 2004년 11월 26일까지의 데이터 저장
for i in range(1, 555):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(i)

    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')

    # 금리 저장 / 상승률 포함
    for i in soup.select('td.num'):
        call_tem.append(i.text)

    # 날짜 저장
    for date_num in soup.select('td.date'):
        date_num = date_num.text
        date_num = date_num.replace("\n", "")
        date_num = date_num.replace("\t", "")
        date_num = date_num.replace(".", "-")
        date_time = datetime.strptime(date_num, '%Y-%m-%d')
        date.append(date_time)

# 금리 저장
for rate in range(len(call_tem)):
    if rate % 3 == 0:
        call_rate.append(call_tem[rate])

len(date), len(call_rate)

In [ ]:
import pandas as pd
call_all = pd.DataFrame(call_rate, index=date, columns=['콜 금리'])
call_all

### **3-2. 기준금리 수집**

In [ ]:
import requests
from bs4 import BeautifulSoup

# 빈 리스트 생성
rate_all = []
std_rate = []
date_year = []
date_month = []

# 크롤링
url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'

res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')

# 그래프 전체 데이터 수집
for data in soup.select('tbody td'):
   rate_all.append(data.text)

# 기준금리 저장
for num in range(len(rate_all)):
    if num % 3 == 2:
        std_rate.append(rate_all[num])

    if num % 3 == 0:
        date_year.append(rate_all[num])

    if num % 3 == 1:
        date_month.append(rate_all[num])

len(std_rate), len(date_year), len(date_month)

In [ ]:
# 월과 일 분리
import re

month = []
day = []

for i in date_month:
    month.append(re.findall('[0-9][0-9]', i)[0])
    day.append(re.findall('[0-9][0-9]', i)[1])

In [ ]:
# 날짜 합치기
std_date = []

for i in range(len(std_rate)):
    date_time = datetime.strptime("{}-{}-{}".format(date_year[i], month[i], day[i]), '%Y-%m-%d')
    std_date.append(date_time)

std_date

In [ ]:
import pandas as pd
call_all = pd.DataFrame(call_rate, index=date, columns=['콜 금리'])
std_all = pd.DataFrame(std_rate, index=std_date, columns=['기준 금리'])

print(call_all, std_all)

In [ ]:
interest_rate = pd.merge(call_all, std_all, how='outer', left_index=True, right_index=True)

In [ ]:
interest_rate.to_csv('/content/interest_rate.csv', encoding='utf-8-sig')